<a href="https://colab.research.google.com/github/Kebrenok/Simulative-/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%92%D1%82%D0%BE%D1%80%D0%BE%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-06-27 17:12:02--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-06-27 17:12:02--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-06-27 17:12:02 (31.2 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('/content/creds.json', scope)

client = gspread.authorize(creds)

worksheets = client.open_by_key("1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U")



**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

Напишите свое решение ниже

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import math


# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)


def generate_report(worksheets):
  sheet1 = worksheets.worksheet("Лист1")
  sheet2 = worksheets.worksheet("Лист2")
  sheet3 = worksheets.worksheet("Лист3")

  sheet1_data = sheet1.get_all_records()
  sheet2_data = sheet2.get_all_records()
  sheet3_data = sheet3.get_all_records()

  res = []

  for r2 in sheet2_data:
    if datetime.strptime(r2['expected_payment_date'], '%d.%m.%Y' ) < datetime.strptime('01.03.2023', '%d.%m.%Y' ): #если дата предстаящего платежа меньше текущей, значит есть просрочка
      days_over = datetime.strptime('01.03.2023', '%d.%m.%Y' ) - datetime.strptime(r2['last_payment_date'], '%d.%m.%Y' ) #считаем дни просрочки
      cnt = math.floor(days_over.days/183) #сколько периодов просрочено
      student_id = r2['student_id']
      name = None
      debt = 0
      for r1 in sheet1_data: # для тех, у кого есть просрочка, ищем имя
        if r1['student_id'] == student_id:
          name = r1['student_name']
      for r3 in sheet3_data: # для просрочников считаем сумму долга
        if r3['student_id'] == student_id:
          debt = cnt*r3['one-time_payment']
          if debt > r3['left_to_pay']:
            debt =  r3['left_to_pay']
      if name and debt > 0:
        res.append(f'Студент {name} - долг {debt} рублей')
  with open('student_debt_report.txt', 'w') as res_f:
    res_f.write('\n'.join(res))

generate_report(worksheets)


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd
generate_report()
user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2025-06-27 17:21:28--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.114.3
Connecting to gist.github.com (gist.github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2025-06-27 17:21:28--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt.1’

student_debt.txt.1  100%[===================>]  11.06K  --.-KB/s    in 0s      

2025-06-27 17:21:29 (22.6 MB/

TypeError: generate_report() missing 1 required positional argument: 'worksheets'

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

AssertionError: При проверке возникла ошибка NameError("name 'user_answer' is not defined")